###Capstone Project-Predicting House Prices

#####Background

A house has many characteristics that describe it from the type of material that is used on the exterior of the house to the number of rooms in the house. Typically, only a few key characteristics are used to describe a house instead of all of the descriptors when describing a house.  A common descriptor of a house is home price because it takes into account the value of many other characteristics. The goal of this project is to predict home prices in Ames, Iowa.  Currently, there is not a published deterministic relationship between home price and the characteristics of a home.  

Predicting home price is important to the buyer, seller, as well as an assessor of the home.  Once home price can be predicted, recommendations can be made to the home seller on what modifications to the home will lead to the greatest increase in price.  An assesor that is determining the value of the home can use the results of this prediction model to verify that their valuation is accurate.  Finally, a home buyer would find it useful to predict home price so that they can verify they are buying the home for a fair value compared to the other homes in Ames, Iowa.  


####Data Wrangling

The dataset was downloaded from https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data.  This dataset is part of a Kaggle competition.  Each row in this dataset represents a house in Ames, Iowa that has 79 features describing the house. The goal is to predict the sale price feature of the test set.  The models will be evaluated using the RMSE between the log of the predicted house sale price and the actual house sale price.  The log scale is used so that more expensive homes do not have a larger weight on the error compared to the less expensive homes.

The Koalas library was used when interacting with big data because it implements the Pandas DataFrame API on top of Apache Spark.  Pandas is typically used for single node Dataframe implementation in Python while Spark is used for big data processing.  With Koalas, Pandas syntax can be used while processing dataframes with Apache Spark.

In [3]:
dbutils.library.installPyPI("koalas")
dbutils.library.restartPython()

PyPI package koalas has been installed already. The previously-installed package is `koalas`. To resolve this issue detach and re-attach the notebook to create a new environment or rename the package.

A sale price feature was created with all of the values in the column set to unknown in the test set. The label in the train and test set was separated from its features so that predicted variable is not mixed in with the features.  Train_label, train_features, test_label, and test_features were created so that operation on the features do not affect the labels.  In addition, the training and test sets were concatenated so that any operations or data exploration on the training set could also be done on the test set.

In [5]:
import databricks.koalas as ks
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
train_csv=pd.read_csv("/dbfs/FileStore/tables/train.csv")
train=ks.from_pandas(train_csv)

train_label=train['SalePrice'].reset_index(drop=True)
train_features=train.drop(['SalePrice'], axis=1)

test_csv=pd.read_csv('/dbfs/FileStore/tables/test.csv')
test=ks.from_pandas(test_csv)

test['SalePrice']='Unknown'
test_label=test['SalePrice'].reset_index(drop=True)
test_features=test.drop(['SalePrice'], axis=1)

frames=[train_features, test_features]
full_df=ks.concat(frames)
full_df.head(5)



,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,None,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,None,None,None,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,None,None,None,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,Ex,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0,42,0,0,0,0,None,None,None,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,Gd,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0,35,272,0,0,0,None,None,None,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192,84,0,0,0,0,None,None,None,0,12,2008,WD,Normal


In [6]:
sns.set_style("white")
sns.set_color_codes(palette='deep')
f, ax = plt.subplots(figsize=(8, 7))
#Check the new distribution 
sns.distplot(train_label.to_numpy(), color="b");
ax.xaxis.grid(False)
ax.set(ylabel="Frequency")
ax.set(xlabel="Sale Price")
ax.set(title="Sale Price Distribution")

display(f)


In [7]:

print("Mean: %f" % train_label.mean())
print("Standard Deviation: %f" % train_label.std())

mu=train_label.mean()
sigma=train_label.std()
interval=stats.norm.interval(0.95, loc=mu, scale=sigma)
print("95% Confidence interval: {}".format(interval))

print("Skewness: %f" % train_label.skew())
print("Kurtosis: %f" % train_label.kurt())


Mean: 180921.195890
Standard Deviation: 79442.502883
95% Confidence interval: (25216.751398233784, 336625.64038258814)
Skewness: 1.880941
Kurtosis: 6.509812

Sale price is not normally distributed as the value of the skewness is above 1.88 which is above 0.  This value for skewness means the sale price is skewed to the right, a distribution that is skewed to the right means the right tail is larger than the left tail.  There are quite a few homes that worth more than 336,625$, which is 2 standard deviations more than the mean. A kurtosis value of 6.5  is larger than 3 which means the distribution is heavy tailed meaning there are more outliers compared to the number of outliers in a normal distribution.

In [9]:
full_df=full_df.drop(['Id'])

The id column is dropped because it does not describe the house and is unique for every row in the dataset.  The numeric variables in the training dataset were graphed in relationship to the predicted variable, house price.

Missing Values

A list of the top 10 features with any missing values was computed from the full_df dataframe which includes data from both train and test.  In addition, a bar plot was constructed of all of the features with missing values and their corresponding percentage of missing values.

In [12]:


def missing_values(full_df):
  missing_value_features={}
  for i in full_df.columns:
    if full_df[i].hasnans:

      missing_value_features[i]=round(full_df[i].isna().sum()/len(full_df[i])*100, 2)


  missing_value_features=sorted(missing_value_features.items(), key=lambda item: item[1], reverse=True)

  print(missing_value_features)
  return missing_value_features

missing_value_features=missing_values(full_df)


[('PoolQC', 99.66), ('MiscFeature', 96.4), ('Alley', 93.22), ('Fence', 80.44), ('FireplaceQu', 48.65), ('LotFrontage', 16.65), ('GarageYrBlt', 5.45), ('GarageFinish', 5.45), ('GarageQual', 5.45), ('GarageCond', 5.45), ('GarageType', 5.38), ('BsmtCond', 2.81), ('BsmtExposure', 2.81), ('BsmtQual', 2.77), ('BsmtFinType2', 2.74), ('BsmtFinType1', 2.71), ('MasVnrType', 0.82), ('MasVnrArea', 0.79), ('MSZoning', 0.14), ('Utilities', 0.07), ('BsmtFullBath', 0.07), ('BsmtHalfBath', 0.07), ('Functional', 0.07), ('Exterior1st', 0.03), ('Exterior2nd', 0.03), ('BsmtFinSF1', 0.03), ('BsmtFinSF2', 0.03), ('BsmtUnfSF', 0.03), ('TotalBsmtSF', 0.03), ('Electrical', 0.03), ('KitchenQual', 0.03), ('GarageCars', 0.03), ('GarageArea', 0.03), ('SaleType', 0.03)]

In [13]:
sns.set_style("white")
sns.set_color_codes(palette='deep')
f, ax = plt.subplots(figsize=(10, 5))
#Check the new distribution 
x=[]
y=[]
for i in missing_value_features:
  x.append(i[0])
  y.append(i[1])
chart=sns.barplot(x=x,y=y,color="b");
ax.xaxis.grid(False)
ax.set(ylabel="Missing Value Percentage")
ax.set(xlabel="Features with Missing Values")
ax.set(title="Missing Value Percentage of each Each Feature")

chart.set_xticklabels(
    chart.get_xticklabels(), 
    rotation=80, 
    horizontalalignment='right',
    fontweight='light',
    fontsize='small'

)

sns.despine(trim=True, left=True)

display(f)

In [14]:
full_df.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,None,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,None,None,None,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,None,None,None,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,Ex,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0,42,0,0,0,0,None,None,None,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,Gd,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0,35,272,0,0,0,None,None,None,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192,84,0,0,0,0,None,None,None,0,12,2008,WD,Normal


Several of the features in full_df are represented as numerical variables even though they should be represented as categorical variables.  For example, the MSSubClass feature is a code that represents the type of dwelling.  Each code is not a measure of a certain characteristic, rather it is a label and should be converted to a string format and treated as a categorical variable. MSSubClass along with MoSold, and YrSold were converted to categorical variables.

In [16]:

full_df['MSSubClass']=full_df['MSSubClass'].apply(str)
full_df['MoSold']=full_df['MoSold'].apply(str)
full_df['YrSold']=full_df['YrSold'].apply(str)



The following function replaces missing values in categorical features with the string 'None'.  In numerical columns, missing values are replaced with mean value of the feature.

In [18]:
def fill_missing_values(full_df):
 
  for i in full_df.columns:
    if full_df[i].hasnans and full_df[i].dtype==('O'):
      
      full_df[i]=full_df[i].fillna('None')
    elif full_df[i].hasnans and full_df[i].dtype!=('O'):
      
      full_df[i]=full_df[i].fillna(full_df[i].mean())
      
  
  return full_df


full_df=fill_missing_values(full_df)

In [19]:
full_df.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,None,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,None,None,None,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,None,None,None,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,Ex,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0,42,0,0,0,0,None,None,None,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,Gd,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0,35,272,0,0,0,None,None,None,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192,84,0,0,0,0,None,None,None,0,12,2008,WD,Normal


In order to verify that all of the missing values in full_df were replaced, the number of features with missing values was counted and the result was that none of the features had any missing values.

In [21]:
len(missing_values(full_df))

[]
Out[11]: 0

### Feature Engineering

Several new features were created such as the number of years since a remodel, the total home quality, the total square footage, the total number of baths, and the total porch square footage.  In addition the binary features has_pool, has2ndfloor, hasbsmt, and hasfireplace were created.  Feature engineering creates more features for the machine learning models to account for and could possible create features that contribute to home price prediction.

In [23]:
full_df['YrSinceRemodel']=(full_df['YrSold']).astype(int)-(full_df['YearRemodAdd']).astype(int)
full_df['Total_Home_Quality']=full_df['OverallQual']+full_df['OverallCond']
full_df['Total_SF']=full_df['TotalBsmtSF']+full_df['1stFlrSF']+full_df['2ndFlrSF']
full_df['Total_bath']=full_df['BsmtFullBath']+0.5*full_df['BsmtHalfBath']+full_df['FullBath']+0.5*full_df['HalfBath']
full_df['Total_porch_sf']=full_df['WoodDeckSF']+full_df['OpenPorchSF']+full_df['EnclosedPorch']+full_df['3SsnPorch']+full_df['ScreenPorch']

full_df['has_pool']=full_df['PoolArea'].apply(lambda x: 1 if x>0 else 0)
full_df['has2ndfloor']=full_df['2ndFlrSF'].apply(lambda x: 1 if x >0 else 0)
full_df['hasbsmt']=full_df['TotalBsmtSF'].apply(lambda x: 1 if x>0 else 0)
full_df['hasfireplace']=full_df['Fireplaces'].apply(lambda x: 1 if x>0 else 0)
full_df['hasgarage'] = full_df['GarageArea'].apply(lambda x: 1 if x > 0 else 0)


#Exploratory Data Analysis Graphs

Below is a graph of house sale price vs total square feet.  The house sale price is trending upwards as total square feet increases.  The slope of the polynomial fit is 75.63 which means an additional square foot costs on average 75.63 dollars.  Since the house price vs. total square feet is a nonuniform distribution, total square feet could be an important predictor in the machine learning phase.

In [26]:
from matplotlib.pyplot import figure
from numpy.polynomial.polynomial import polyfit

figure(num=None, figsize=(10, 5), dpi=80, facecolor='w', edgecolor='k')


a=plt.subplot(1,1,1)
x=full_df[0:len(train_features)]['Total_SF']
y=train_label
ax.xaxis.labelpad = 50
plt.scatter(x.to_numpy(), y.to_numpy())
plt.title('House Sale Price vs. Total_SF')
plt.xlabel('Total_SF')
plt.ylabel('House Sale Price')

b, m = polyfit(x.to_numpy(), y.to_numpy(), 1)
plt.plot(x.to_numpy(), b + m * x.to_numpy(), '-', color='r')

display(a)
print('m=75.63')

Below is a graph of house sale price vs total bathrooms. The house sale price is trending upwards as total bathrooms increases. The slope of the polynomial fit is 63899.15 which means an additional bathroom costs on average 63899.15 dollars. Since the house price vs. total bathrooms is a nonuniform distribution, total bathrooms could be an important predictor in the machine learning phase.

In [28]:
from matplotlib.pyplot import figure
from numpy.polynomial.polynomial import polyfit

figure(num=None, figsize=(10, 5), dpi=80, facecolor='w', edgecolor='k')


a=plt.subplot(1,2,1)
x=full_df[0:len(train_features)]['Total_bath']
y=train_label
ax.xaxis.labelpad = 50
plt.scatter(x.to_numpy(), y.to_numpy())
plt.title('House Sale Price vs. Total_bath')
plt.xlabel('Total_bath')
plt.ylabel('House Sale Price')

b, m = polyfit(x.to_numpy(), y.to_numpy(), 1)
plt.plot(x.to_numpy(), b + m * x.to_numpy(), '-', color='r')

display(a)
print('m=63899.15')

Below is a graph of house sale price vs year built. The house sale price is trending upwards as year built increases. The slope of the polynomial fit is 1375.37. Since the house price vs. year built is a nonuniform distribution, year built could be an important predictor in the machine learning phase.

In [30]:
from matplotlib.pyplot import figure
from numpy.polynomial.polynomial import polyfit

figure(num=None, figsize=(10, 5), dpi=80, facecolor='w', edgecolor='k')


a=plt.subplot(1,2,1)
x=full_df[0:len(train_features)]['YearBuilt']
y=train_label
ax.xaxis.labelpad = 50
plt.scatter(x.to_numpy(), y.to_numpy())
plt.title('House Sale Price vs. YearBuilt')
plt.xlabel('YearBuilt')
plt.ylabel('House Sale Price')

b, m = polyfit(x.to_numpy(), y.to_numpy(), 1)
plt.plot(x.to_numpy(), b + m * x.to_numpy(), '-', color='r')

display(a)
print('m=1375.37')

d Below is a graph of house sale price vs number of bedrooms. The house sale price is trending upwards as number of bedrooms increases increases. The slope of the polynomial fit is 16381.02. Since the house price vs. number of bedrooms is a nonuniform distribution, year built could be an important predictor in the machine learning phase.

In [32]:
from matplotlib.pyplot import figure
from numpy.polynomial.polynomial import polyfit

figure(num=None, figsize=(10, 5), dpi=80, facecolor='w', edgecolor='k')


a=plt.subplot(1,2,1)
x=full_df[0:len(train_features)]['BedroomAbvGr']
y=train_label
ax.xaxis.labelpad = 50
plt.scatter(x.to_numpy(), y.to_numpy())
plt.title('House Sale Price vs. BedroomAbvGr')
plt.xlabel('BedroomAbvGr')
plt.ylabel('House Sale Price')

b, m = polyfit(x.to_numpy(), y.to_numpy(), 1)
plt.plot(x.to_numpy(), b + m * x.to_numpy(), '-', color='r')

display(a)
print('m=16381.02')

Below is a graph of house sale price vs bulding type. Since the house price vs. building type is a nonuniform distribution, building type could be an important predictor in the machine learning phase.

In [34]:
from matplotlib.pyplot import figure



a=plt.subplot(1,2,1)
x=full_df[0:len(train_features)]['BldgType']
y=train_label
frames=[x,y]
output=ks.concat(frames, axis=1)
ax.xaxis.labelpad = 50
output=output.groupby('BldgType').mean()['SalePrice']
print(output)
plt.bar(output.index.to_numpy(), output.to_numpy())
plt.title('House Sale Price vs. BldgType')
plt.xlabel('BldgType')
plt.ylabel('House Sale Price')
a.set_xticklabels(output.index.to_numpy(), rotation = 45)
display(a)


The features with numeric and string data types are separated and counted below.

In [36]:
def get_counts(full_df):

  numeric=[]
  string=[]
  counter_numeric=0
  counter_string=0
  for i in full_df.columns:
    if full_df[i].dtype!='O':
      numeric.append(i)
      counter_numeric+=1
    if full_df[i].dtype=='O':
      string.append(i)
      counter_string+=1
    
  
  print(counter_numeric, counter_string)
  return numeric, string


numeric, string=get_counts(full_df)
   
    

43 46

Below are scatter plots of all of the numeric features vs House Price.

In [38]:


def graph(full_df):
  figure(figsize=(50, 150))
  count=0
  for i in numeric:
      count+=1
      fig=plt.subplot(15,3,count)
      x=full_df[0:len(train_features)][i]
      y=train_label
      plt.scatter(x.to_numpy(), y.to_numpy())
      plt.title('House Sale Price vs. {}'.format(i), size=25)
      plt.xlabel(i, size=25)
      plt.ylabel('House Sale Price', size=25)
 
  display(fig)

   
     
  
graph(full_df)


Below are bar plots of all of the string features vs House Price.

In [40]:
def graph2(full_df):
  figure(figsize=(50, 150))
  count=0   
  for i in string:
        count+=1
        fig2=plt.subplot(16,3,count)
        x=full_df[0:len(train_features)][i]
        y=train_label
        frames=[x,y]
        output=ks.concat(frames, axis=1)
        output=output.groupby(i).mean()['SalePrice']
        plt.bar(output.index.to_numpy(), output.to_numpy())
        plt.title('House Sale Price vs. {}'.format(i), size=25)
        plt.xlabel(i, size=25)
        plt.ylabel('House Sale Price', size=25)

  display(fig2)
    

graph2(full_df)

A dashboard was created of the pertinent features for prediction.  It can be viewed in the dashboard viewer of databricks.

Important Features Dashboard

Categorical variables were hot one encoded so that a machine learning model can take in these features.  Each unique value in a column gets a new column in which if the row contains the unique value a value of 1 is used else a value of 0 is used.  Hot one encoding increases the number of features from 87 to 349.

In [44]:
all_features=full_df.toPandas()

all_features.MSSubClass=all_features.MSSubClass.astype(str)
all_features.MoSold=all_features.MoSold.astype(str)
all_features.YrSold=all_features.YrSold.astype(str)

output_df=pd.get_dummies(all_features).reset_index(drop=True)

print(output_df.shape)
output_df.head(5)

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,YrSinceRemodel,Total_Home_Quality,Total_SF,Total_bath,Total_porch_sf,has_pool,has2ndfloor,...,Fence_MnWw,Fence_None,MiscFeature_Gar2,MiscFeature_None,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,MoSold_1,MoSold_10,MoSold_11,MoSold_12,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,YrSold_2006,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_None,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,65.0,8450,7,5,2003,2003,196.0,706.0,0.0,150.0,856.0,856,854,0,1710,1.0,0.0,2,1,3,1,8,0,2003.0,2.0,548.0,0,61,0,0,0,0,0,5,12,2566.0,3.5,61,0,1,...,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,80.0,9600,6,8,1976,1976,0.0,978.0,0.0,284.0,1262.0,1262,0,0,1262,0.0,1.0,2,0,3,1,6,1,1976.0,2.0,460.0,298,0,0,0,0,0,0,31,14,2524.0,2.5,298,0,0,...,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,68.0,11250,7,5,2001,2002,162.0,486.0,0.0,434.0,920.0,920,866,0,1786,1.0,0.0,2,1,3,1,6,1,2001.0,2.0,608.0,0,42,0,0,0,0,0,6,12,2706.0,3.5,42,0,1,...,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,60.0,9550,7,5,1915,1970,0.0,216.0,0.0,540.0,756.0,961,756,0,1717,1.0,0.0,1,0,3,1,7,1,1998.0,3.0,642.0,0,35,272,0,0,0,0,36,12,2473.0,2.0,307,0,1,...,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
4,84.0,14260,8,5,2000,2000,350.0,655.0,0.0,490.0,1145.0,1145,1053,0,2198,1.0,0.0,2,1,4,1,9,1,2000.0,3.0,836.0,192,84,0,0,0,0,0,8,13,3343.0,3.5,276,0,1,...,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [45]:

output_df.to_csv('/dbfs/FileStore/cleaned.csv')
all_features.to_csv('/dbfs/FileStore/cleaned1.csv')
train_label=train_label.toPandas()
train_label.to_csv('/dbfs/FileStore/trainlabel.csv', header=False)
